In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import json
# import dataset
import credentials
import sqlite3
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pickle
from Naive_bayes_model import *
import xgboost as xgb

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
f = open('nb1.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [3]:
f1 = open('xgb', 'rb')
XGBClassifier = pickle.load(f1)
f1.close()

In [4]:
conn = sqlite3.connect('tweet2.db')
c = conn.cursor()

In [5]:
def pred(data):
    
    tweetProcessor = PreProcessTweets()
    preprocessedTrainingSet = tweetProcessor.processTweets(data)
    
    word_features = buildVocabulary(preprocessedTrainingSet)
#     print("\n",word_features,"\n")
#     buildVocabulary(preprocessedTrainingSet)
    trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
    
    print(preprocessedTrainingSet[0][0])
    
    NB = NBayesClassifier.classify(extract_features(preprocessedTrainingSet[0][0],word_features))
    
    return NB
    

In [6]:

def attributeselection(tweet,query_word):
    attr= []

    #text = "Alleged East Bay serial arsonist arrested #SanFrancisco - http://t.co/ojuHfkHVb2"
    tweet = tweet.lower() # convert text to lower-case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove username
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    tweet = re.sub(r'\\s+',r' ', tweet, flags=re.I)
    tweet = re.sub("([^\x00-\x7F])+"," ",tweet) #remove all non-english characters
    tweet = re.sub(r"\s+"," ", tweet, flags = re.I) #remove multiple spaces
    text = tweet
    text = nltk.word_tokenize(text)
    poslist = nltk.pos_tag(text)
#     print(poslist)

    verblist = []
    for i in range(len(poslist)):
        if poslist[i][1].startswith('V'):
            verblist.append(poslist[i])
    
    word_count = len(poslist)
#     print(word_count)

    if len(verblist) == 0:
        l = (query_word,'VB')
        verblist.append(l)
    
#     print(verblist,"\n")
    verb_count = len(verblist)
#     print(verb_count)

   # query_word = 'arson'

    words_before = words_after = 0
    for i in range(len(poslist)):
        if poslist[i][0].startswith(query_word):
            words_before = i

    words_after = word_count - words_before - 1

#     print(words_before)
#     print(words_after)
    verbs = []    
    attr.append(word_count)
#     for i in verblist:
#         print(i[0])
#         verbs.append(i[0])
#     attr.append(verbs)
    attr.append(verb_count)
    attr.append(words_before + 1)
#     attr_query_word.append(query_word)
    attr.append(words_before)
    attr.append(words_after)
    
    
        
    column_names = ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
    print(attr,column_names)
    df = pd.DataFrame([attr], columns=column_names)
    arr_x=df.to_numpy(dtype=object)
    print(arr_x,arr_x.shape)
    y_pred = XGBClassifier.predict(arr_x)
    print("PRIORITY: ",y_pred)
    
    

In [7]:
class TwitterStreamer():
    
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
        
        stream = Stream(auth, listener,tweet_mode='extended')
        
        stream.filter(languages=["en"],track=['coronavirus'])
        
 
class StdOutListener(StreamListener):
    
    def on_status(self, status):
        print(status.text)

    def on_error(self, status):
        print(status)

    def on_data(self, data):
        all_data             = json.loads(data)
        created_at           = all_data['created_at']
        favorite_count       = all_data['favorite_count']
        favorited            = all_data['favorited']
        filter_level         = all_data['filter_level']
        lang                 = all_data['lang']
        retweet_count        = all_data['retweet_count']
        retweeted            = all_data['retweeted']
        source               = all_data['source']
        text                 = all_data['text']
        truncated            = all_data['truncated']
        user_created_at      = all_data['user']['created_at']
        user_followers_count = all_data['user']['followers_count']
        user_location        = all_data['user']['location']
        user_lang            = all_data['user']['lang']
        user_name            = all_data['user']['name']
        user_screen_name     = all_data['user']['screen_name']
        user_time_zone       = all_data['user']['time_zone']
        user_utc_offset      = all_data['user']['utc_offset']
        user_friends_count   = all_data['user']['friends_count']
        geo_enabled          = all_data['user']['geo_enabled']  
        coordinates          = all_data['coordinates']
        geo                  = all_data['geo']
        place                = all_data['place']
        if place is not None:
            place_name       = all_data['place']['name']
        else:
            place_name       = all_data['place']

        print("Tweet: "+text+"\n")
        
        data = [{'text': text,'Classfication': ''}]
        
        info = pred(data)
        
        if info == 'Relevant' or info == 1 or info == '1':
            attributeselection(text,'coronavirus')
            
        print("\n Class: "+info+"\n")
                         
        c.execute('''INSERT INTO tweets 
        (created_at, favorite_count, favorited, filter_level, lang, 
                         retweet_count, retweeted, source, text, truncated, user_created_at,  
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ) 
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
            (created_at, favorite_count, favorited, filter_level, lang, retweet_count, 
                         retweeted, source, text, truncated, user_created_at, 
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ))
        
        c.execute('''INSERT INTO tweet_class 
        (text, class) 
            VALUES (?,?)''', 
            (text, info))
        
        conn.commit()

In [8]:
if __name__ == '__main__':
    
    hash_tag_list = ['coronavirus']
    fetched_tweets_filename = "tweets.json"
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

Tweet: RT @ThePlumLineGS: Kellyanne Conway just previewed the Big Lie that Trump will use to salvage reelection amid his coronavirus fiasco.

It r…

['rt', 'kellyanne', 'conway', 'previewed', 'big', 'lie', 'trump', 'use', 'salvage', 'reelection', 'amid', 'coronavirus', 'fiasco', 'r…']
[23, 4, 19, 18, 4] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[23 4 19 18 4]] (1, 5)
PRIORITY:  [0]

 Class: 1



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: RT @ChrisJZullo: @BarackObama Donald Trump knew in January dangers of Coronavirus. Didn’t buy, produce mass tests. Didn’t push social dista…

['rt', 'donald', 'trump', 'knew', 'january', 'dangers', 'coronavirus', '’', 'buy', 'produce', 'mass', 'tests', '’', 'push', 'social', 'dista…']

 Class: 0

Tweet: RT @SAL3977: Please lord have mercy!! 🙏🏼🙏🏼
Field hospital in Central Park ....
#coronavirus #NYC #COVID19 #CentralPark https://t.co/IGTuPAD…

['rt', 'please', 'lord', 'mercy', '🙏🏼🙏🏼', 'field', 'hospital', 'central', 'park', '...', 'coronavirus', 'nyc', 'covid19', 'centralpark']
[20, 3, 16, 15, 4] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[20 3 16 15 4]] (1, 5)
PRIORITY:  [0]

 Class: 1



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: RT @ScramNews: Gove and Hancock said something that was blatantly false.
https://t.co/Lj3wBQJPW9

['rt', 'gove', 'hancock', 'said', 'something', 'blatantly', 'false']

 Class: 0

Tweet: RT @silverncopper: Treasury Secretary Mnuchin says ‘Americans need cash now;’ lawmakers in both parties urge government to send funds to ho…

['rt', 'treasury', 'secretary', 'mnuchin', 'says', '‘', 'americans', 'need', 'cash', '’', 'lawmakers', 'parties', 'urge', 'government', 'send', 'funds', 'ho…']
[22, 5, 1, 0, 21] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[22 5 1 0 21]] (1, 5)
PRIORITY:  [0]

 Class: 1



...

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: Points for creativity! 🤣🤣🤣https://t.co/1Cm3FIFuAw

['points', 'creativity', '🤣🤣🤣URL']

 Class: 0

Tweet: RT @AdamWagner1: I have recorded an episode of @BHumanPodcast with @davidallengreen taking a detailed look at the Coronavirus Regulations a…

['rt', 'recorded', 'episode', 'taking', 'detailed', 'look', 'coronavirus', 'regulations', 'a…']

 Class: 0

Tweet: RT @Thursfields: Furlough leave – the government releases further guidance - https://t.co/VThgvm5NqD https://t.co/yg4uUlA1zU

['rt', 'furlough', 'leave', '–', 'government', 'releases', 'guidance']

 Class: 0

Tweet: RT @TeaPainUSA: Lyin' about the dead?  Can MAGA get any sicker?  

https://t.co/IW1ungQh2X

['rt', 'lyin', 'dead', 'maga', 'get', 'sicker']
[15, 2, 1, 0, 14] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[15 2 1 0 14]] (1, 5)
PRIORITY:  [0]

 Class: 1



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: Great tips for keeping your devices a bit safer in the age of #CoronaVirus #CoronaVirusLockdown, from #@Fox29Philly https://t.co/B86QqNAyeL

['great', 'tips', 'keeping', 'devices', 'bit', 'safer', 'age', 'coronavirus', 'coronaviruslockdown']

 Class: 0



KeyError: 'created_at'